In [ ]:
import pandas as pd
import numpy as np
from io import StringIO

<H4> Files needed: <br>
    * Students_export.csv <br>
    * Schools_export.csv
    

In [ ]:
AZStudents = pd.read_csv("Students_export.csv", encoding='latin-1')

In [ ]:
AZ_FocusedColumns = AZStudents.filter(['STUDENTS.SchoolID', 'STUDENTS.State_StudentNumber', 'STUDENTS.Student_Number', 'STUDENTS.Last_Name', 'STUDENTS.First_Name', 'STUDENTS.Middle_Name', 'STUDENTS.Grade_Level', 'STUDENTS.DOB', 
                                       'STUDENTS.Home_Room', 'STUDENTS.Gender'])

In [ ]:
AZ_FocusedColumns

In [ ]:
transform_schoolID = {
    100:'Maricopa',
    150: 'Casa Grande',
    200: 'Queen Creek',
    250: 'Avondale',
    300: 'Chandler',
    350: 'Northwest Tucson',
    400: 'Laveen',
    450: 'Gilbert',
    500: 'Surprise',
    550: 'North Chandler',
    600: 'Glendale',
    650: 'Peoria',
    700: 'East Mesa',
    750: 'Phoenix',
    800: 'Goodyear',
    850: 'North Phoenix',
    900: 'West Surprise',
    950: 'East Tucson',
    1000: 'Deer Valley',
    1050: 'Mesa',
    1100: 'Legacy Online Academy',
}

AZ_FocusedColumns['School_Bldg'] = AZ_FocusedColumns['STUDENTS.SchoolID'].map(transform_schoolID)

In [ ]:
AZ_FocusedColumns.School_Bldg.value_counts().sort_index()

In [ ]:
AZ_FocusedColumns['District'] = 'Legacy Traditional Schools- AZ'

In [ ]:
AZ_FocusedColumns['School_BldgCode'] = 'LTS-AZ-' + AZ_FocusedColumns['STUDENTS.SchoolID'].astype(str)

In [ ]:
HomeroomStrip = AZ_FocusedColumns['STUDENTS.Home_Room'].str.rstrip(', \n')

In [ ]:
AZ_Partition = AZ_FocusedColumns

In [ ]:
AZ_Partition['HomeRoom_Last'] = AZ_Partition['STUDENTS.Home_Room'].str.partition(', ')[0]

In [ ]:
AZ_Partition['Class'] =  AZ_Partition['STUDENTS.Grade_Level'].astype(str) + ' - ' + AZ_Partition['HomeRoom_Last']

In [ ]:
AZ_Partition['Unique_Student_ID'] = 'AZ' + AZ_Partition['STUDENTS.Student_Number'].astype(str)

In [ ]:
AZ_Partition['format_DOB'] = pd.to_datetime(AZ_Partition['STUDENTS.DOB'])

In [ ]:
AZ_Partition['Date_Of_Birth'] = AZ_Partition['format_DOB'].dt.strftime('%m/%d/%Y')

In [ ]:
AZ_Partition['Code'] = '' 

In [ ]:
AZ_Partition['Active User'] = ''

In [ ]:
AZ_Partition

In [ ]:
AZ_Partition

<H5> Merge on SchoolID now

In [ ]:
# Link school information on School/Building Code
School_Info = pd.read_csv("Schools_export.csv", encoding='latin-1')

In [ ]:
df_SchoolAddresses = School_Info.filter(['PS School Number', 'SCHOOLS.Name', 'SchoolAddress', 'SchoolCity', 'SchoolState', 'SchoolZip', 'STUDENTS.Grade_Level', 'STUDENTS.DOB']).dropna() 


In [ ]:
df_SchoolAddresses

In [ ]:
df_MergeTest = AZ_Partition.merge(df_SchoolAddresses, left_on='STUDENTS.SchoolID',right_on='PS School Number')

In [ ]:
df_MergeTest

In [ ]:
AZ_FilteredColumns = AZ_Partition.filter(['District','School_BldgCode', 'Class', 'Code', 'STUDENTS.First_Name', 
'STUDENTS.Last_Name', 'STUDENTS.Middle_Name', 'Unique_Student_ID','Date_Of_Birth', 'STUDENTS.Gender', 'STUDENTS.Grade_Level', 'Active User', 'School_Bldg', 'STUDENTS.State_StudentNumber' ])

In [ ]:
df_MergeTest.info()

In [ ]:
df_MergeTest

In [ ]:
AZ_FilteredLocations = df_MergeTest.filter(['District', 'School_Bldg', 'School_BldgCode', 'Class', 'STUDENTS.Grade_Level', 
                                            'Code', 'SchoolAddress', 'SchoolCity', 'SchoolState', 'SchoolZip'])

In [ ]:
AZ_FilteredLocations

In [ ]:
AZ_FilteredColumns = AZ_Partition.filter(['District','School_Bldg', 'School_BldgCode', 'Class', 'Code', 'STUDENTS.First_Name', 
'STUDENTS.Last_Name', 'STUDENTS.Middle_Name', 'Unique_Student_ID','Date_Of_Birth', 'STUDENTS.Gender', 'STUDENTS.Grade_Level', 'Active User', 'STUDENTS.State_StudentNumber'])

In [ ]:
AZ_FilteredColumns

In [ ]:
AZ_FilteredColumns.info()

In [ ]:
AZ_FilteredLocations

In [ ]:
dict = {'District' : 'District/Area *',
        'School_Bldg' : 'School/Building *',
        'School_BldgCode' : 'School/Building Code',
        'Class' : 'Class *',
        'STUDENTS.Grade_Level' : 'Grade *',
        'SchoolAddress' : 'Address',
        'SchoolCity' : 'City',
        'SchoolState' : 'State',
        'SchoolZip' : 'Zip',
       }
AZ_FilteredLocations.rename(columns=dict, inplace=True)

In [ ]:
AZ_FilteredLocations

In [ ]:
AZ_LocationsToWrite = AZ_FilteredLocations.drop_duplicates()

In [ ]:
AZ_LocationsToWrite

In [ ]:
AZ_LocationsToWrite.to_csv('AZ_CoGAT_location.csv', index=False, encoding='utf-8')

In [ ]:
dict = {'District' : 'District/Area *',
        'School_Bldg' : 'School/Building *',
        'School_BldgCode' : 'School/Building Code *',
        'Class' : 'Class *',
        'STUDENTS.First_Name' : 'First Name *',
        'STUDENTS.Last_Name' : 'Last Name *',
        'STUDENTS.Middle_Name' : 'Middle Name',
        'Unique_Student_ID' : 'Unique Student ID *',
        'Date_Of_Birth' : 'Date of Birth *',
        'STUDENTS.Gender' : 'Gender *',
        'STUDENTS.Grade_Level' : 'Grade *',
        'STUDENTS.State_StudentNumber' : 'Additional ID Number'
       }
AZ_FilteredColumns.rename(columns=dict, inplace=True)

In [ ]:
AZ_FilteredColumns

In [ ]:
AZ_FilteredColumns.info()

In [ ]:
AZ_FilteredColumns.to_csv('AZ_CoGAT_student.csv', index=False, encoding='utf-8')